In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [5]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels = zip(*data)

## Inspect label counts

In [6]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [7]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels = zip(*balanced_data)

In [8]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


# Prepare 10 cross validation folds

In [9]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [10]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [11]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [12]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [13]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [14]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [15]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [17]:
#class LangIdConvNet(nn.Module):
#    def __init__(self):
#        super(LangIdConvNet, self).__init__()
#        
#        self.lstm = nn.LSTM(input_size=512, hidden_size=3, num_layers=1, batch_first=True)
#        # self.gru = nn.GRU(input_size=512, hidden_size=3, num_layers=1, batch_first=True)
#        self.drop1 = nn.Dropout(p=0.3)
#        self.lin1 = nn.Linear(in_features=3, out_features=3)
        
        
        
#    def forward(self, x):
#        output, (h_n, c_n) = self.lstm(x)
#        # output, h_n = self.gru(x)
#        x = output[:, -1, :] # [all batches, last time step, all neurons]
#        
#        x = self.drop1(x)
#        x = self.lin1(x)
#        
#        return x

class LangIdRNN(nn.Module):
    def __init__(self):
        super(LangIdRNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        # x = self.drop2(x)
        x = self.lin2(x)
        
        return x

# Train Classification Models

In [18]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    
    return train_x, train_y, test_x, test_y

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y = get_data_for_fold(fold_id, feature_name, batch_size)
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [19]:
def train_on_fold(fold_id, feature_name, batch_size, epochs):
    device = torch.device("cuda:0")
    torch.manual_seed(0)
    results = {}
    
    model = LangIdRNN().to(device)

    train_loader, test_loader = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(LangIdRNN(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc
        }
        
    return results
    

In [20]:
import csv
from pathlib import Path

def save_results(all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        Path('lstm_results_3_fixed').mkdir(exist_ok=True)
        fname = f"lstm_results_3_fixed/{feature_name}_{fold_index}.csv"
        with open(fname, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(
                ['epoch', 'train_loss', 'test_loss', 'train_acc', 'test_acc']
            )
            
            for epoch in sorted(result_entry['epochs'].keys()):
                writer.writerow([
                    epoch,
                    result_entry['epochs'][epoch]['train_loss'],
                    result_entry['epochs'][epoch]['test_loss'],
                    result_entry['epochs'][epoch]['train_acc'],
                    result_entry['epochs'][epoch]['test_acc']
                ])

In [ ]:
all_results = []

for fold_index in cv_folds:
    for feature_name in raw_features:
        resutls = train_on_fold(fold_index, feature_name, batch_size=100, epochs=1000)
        all_results.append({
            'fold_index': fold_index,
            'feature_name': feature_name,
            'epochs': resutls
        })
        save_results(all_results)

---------------------------------------------------------------------------------
      Layer (type)                  Output Shape         Param #     Tr. Param #
          Conv1d-1                 [10, 1, 2996]           1,537           1,537
         Dropout-2                 [10, 1, 2996]               0               0
       MaxPool1d-3                 [10, 1, 1498]               0               0
             GRU-4     [10, 1498, 3], [1, 10, 3]              54              54
          Linear-5                       [10, 3]              12              12
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
---------------------------------------------------------------------------------
Epoch: 10. Train Loss: 1.136. Test Loss: 1.135. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 20. Train Loss: 1.108. Test Loss: 1.13. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 30. Train Loss: 1.086. Test Loss: 1.126. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 40. Train Loss: 1.063. Test

Epoch: 890. Train Loss: 0.1198. Test Loss: 1.929. Train Acc: 1.0. Test Acc:0.2727
Epoch: 900. Train Loss: 0.1089. Test Loss: 1.976. Train Acc: 1.0. Test Acc:0.2727
Epoch: 910. Train Loss: 0.1274. Test Loss: 1.991. Train Acc: 0.9804. Test Acc:0.2727
Epoch: 920. Train Loss: 0.09547. Test Loss: 2.007. Train Acc: 1.0. Test Acc:0.2727
Epoch: 930. Train Loss: 0.1079. Test Loss: 1.943. Train Acc: 1.0. Test Acc:0.3333
Epoch: 940. Train Loss: 0.09228. Test Loss: 1.99. Train Acc: 1.0. Test Acc:0.2727
Epoch: 950. Train Loss: 0.1051. Test Loss: 1.98. Train Acc: 0.9804. Test Acc:0.303
Epoch: 960. Train Loss: 0.09942. Test Loss: 1.983. Train Acc: 1.0. Test Acc:0.2727
Epoch: 970. Train Loss: 0.1092. Test Loss: 1.995. Train Acc: 1.0. Test Acc:0.303
Epoch: 980. Train Loss: 0.1054. Test Loss: 2.024. Train Acc: 1.0. Test Acc:0.2727
Epoch: 990. Train Loss: 0.09427. Test Loss: 1.995. Train Acc: 1.0. Test Acc:0.3333
Epoch: 1000. Train Loss: 0.08409. Test Loss: 2.063. Train Acc: 1.0. Test Acc:0.2727
--------

Epoch: 770. Train Loss: 0.136. Test Loss: 1.713. Train Acc: 1.0. Test Acc:0.4545
Epoch: 780. Train Loss: 0.1427. Test Loss: 1.718. Train Acc: 0.9804. Test Acc:0.4545
Epoch: 790. Train Loss: 0.1929. Test Loss: 1.72. Train Acc: 0.9608. Test Acc:0.4545
Epoch: 800. Train Loss: 0.1216. Test Loss: 1.736. Train Acc: 1.0. Test Acc:0.4848
Epoch: 810. Train Loss: 0.1678. Test Loss: 1.781. Train Acc: 0.9804. Test Acc:0.4242
Epoch: 820. Train Loss: 0.1126. Test Loss: 1.754. Train Acc: 1.0. Test Acc:0.4242
Epoch: 830. Train Loss: 0.1166. Test Loss: 1.765. Train Acc: 1.0. Test Acc:0.4242
Epoch: 840. Train Loss: 0.2053. Test Loss: 1.79. Train Acc: 0.9804. Test Acc:0.4242
Epoch: 850. Train Loss: 0.1296. Test Loss: 1.818. Train Acc: 1.0. Test Acc:0.4242
Epoch: 860. Train Loss: 0.1141. Test Loss: 1.824. Train Acc: 1.0. Test Acc:0.4242
Epoch: 870. Train Loss: 0.1098. Test Loss: 1.853. Train Acc: 1.0. Test Acc:0.4545
Epoch: 880. Train Loss: 0.1217. Test Loss: 1.851. Train Acc: 1.0. Test Acc:0.4242
Epoch: 

Epoch: 650. Train Loss: 0.1714. Test Loss: 1.348. Train Acc: 1.0. Test Acc:0.4848
Epoch: 660. Train Loss: 0.152. Test Loss: 1.356. Train Acc: 1.0. Test Acc:0.4848
Epoch: 670. Train Loss: 0.1653. Test Loss: 1.366. Train Acc: 1.0. Test Acc:0.5152
Epoch: 680. Train Loss: 0.1613. Test Loss: 1.381. Train Acc: 1.0. Test Acc:0.4848
Epoch: 690. Train Loss: 0.1672. Test Loss: 1.386. Train Acc: 1.0. Test Acc:0.4848
Epoch: 700. Train Loss: 0.1573. Test Loss: 1.407. Train Acc: 1.0. Test Acc:0.4848
Epoch: 710. Train Loss: 0.1465. Test Loss: 1.406. Train Acc: 1.0. Test Acc:0.4848
Epoch: 720. Train Loss: 0.1297. Test Loss: 1.428. Train Acc: 1.0. Test Acc:0.4848
Epoch: 730. Train Loss: 0.1258. Test Loss: 1.438. Train Acc: 1.0. Test Acc:0.4848
Epoch: 740. Train Loss: 0.1261. Test Loss: 1.439. Train Acc: 1.0. Test Acc:0.4848
Epoch: 750. Train Loss: 0.135. Test Loss: 1.463. Train Acc: 1.0. Test Acc:0.4848
Epoch: 760. Train Loss: 0.1254. Test Loss: 1.453. Train Acc: 1.0. Test Acc:0.4545
Epoch: 770. Train 

Epoch: 530. Train Loss: 0.249. Test Loss: 1.402. Train Acc: 0.9804. Test Acc:0.3636
Epoch: 540. Train Loss: 0.2396. Test Loss: 1.423. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 550. Train Loss: 0.2109. Test Loss: 1.432. Train Acc: 1.0. Test Acc:0.3333
Epoch: 560. Train Loss: 0.1944. Test Loss: 1.441. Train Acc: 1.0. Test Acc:0.3333
Epoch: 570. Train Loss: 0.1915. Test Loss: 1.459. Train Acc: 1.0. Test Acc:0.3333
Epoch: 580. Train Loss: 0.203. Test Loss: 1.474. Train Acc: 1.0. Test Acc:0.3333
Epoch: 590. Train Loss: 0.1957. Test Loss: 1.482. Train Acc: 0.9804. Test Acc:0.303
Epoch: 600. Train Loss: 0.1835. Test Loss: 1.508. Train Acc: 1.0. Test Acc:0.3636
Epoch: 610. Train Loss: 0.1921. Test Loss: 1.521. Train Acc: 0.9804. Test Acc:0.303
Epoch: 620. Train Loss: 0.1898. Test Loss: 1.533. Train Acc: 0.9804. Test Acc:0.303
Epoch: 630. Train Loss: 0.1648. Test Loss: 1.544. Train Acc: 1.0. Test Acc:0.3333
Epoch: 640. Train Loss: 0.1794. Test Loss: 1.562. Train Acc: 1.0. Test Acc:0.3333
Epoch:

...

Epoch: 530. Train Loss: 0.2183. Test Loss: 1.715. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 540. Train Loss: 0.2365. Test Loss: 1.726. Train Acc: 0.9412. Test Acc:0.303
Epoch: 550. Train Loss: 0.2012. Test Loss: 1.744. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 560. Train Loss: 0.1836. Test Loss: 1.738. Train Acc: 1.0. Test Acc:0.3333
Epoch: 570. Train Loss: 0.1781. Test Loss: 1.751. Train Acc: 1.0. Test Acc:0.3333
Epoch: 580. Train Loss: 0.1901. Test Loss: 1.769. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 590. Train Loss: 0.1814. Test Loss: 1.791. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 600. Train Loss: 0.1745. Test Loss: 1.813. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 610. Train Loss: 0.1656. Test Loss: 1.844. Train Acc: 1.0. Test Acc:0.3333
Epoch: 620. Train Loss: 0.1822. Test Loss: 1.84. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 630. Train Loss: 0.1534. Test Loss: 1.866. Train Acc: 1.0. Test Acc:0.3333
Epoch: 640. Train Loss: 0.183. Test Loss: 1.886. Train Acc: 0.9412. Test Acc:0.

Epoch: 410. Train Loss: 0.3812. Test Loss: 1.772. Train Acc: 0.9804. Test Acc:0.1818
Epoch: 420. Train Loss: 0.3923. Test Loss: 1.812. Train Acc: 0.9804. Test Acc:0.1818
Epoch: 430. Train Loss: 0.3543. Test Loss: 1.819. Train Acc: 0.9804. Test Acc:0.2121
Epoch: 440. Train Loss: 0.3951. Test Loss: 1.836. Train Acc: 0.9216. Test Acc:0.2121
Epoch: 450. Train Loss: 0.3401. Test Loss: 1.855. Train Acc: 0.9608. Test Acc:0.2121
Epoch: 460. Train Loss: 0.3031. Test Loss: 1.908. Train Acc: 1.0. Test Acc:0.2121
Epoch: 470. Train Loss: 0.332. Test Loss: 1.932. Train Acc: 0.9412. Test Acc:0.2121
Epoch: 480. Train Loss: 0.3156. Test Loss: 1.953. Train Acc: 0.9412. Test Acc:0.2121
Epoch: 490. Train Loss: 0.2889. Test Loss: 1.966. Train Acc: 0.9608. Test Acc:0.2121
Epoch: 500. Train Loss: 0.3106. Test Loss: 2.025. Train Acc: 0.9216. Test Acc:0.2121
Epoch: 510. Train Loss: 0.3054. Test Loss: 2.07. Train Acc: 0.9216. Test Acc:0.2121
Epoch: 520. Train Loss: 0.2573. Test Loss: 2.06. Train Acc: 0.9804. Te

Epoch: 290. Train Loss: 0.5528. Test Loss: 1.215. Train Acc: 0.9608. Test Acc:0.3333
Epoch: 300. Train Loss: 0.5251. Test Loss: 1.235. Train Acc: 0.9608. Test Acc:0.3333
Epoch: 310. Train Loss: 0.5366. Test Loss: 1.257. Train Acc: 0.9412. Test Acc:0.303
Epoch: 320. Train Loss: 0.5087. Test Loss: 1.271. Train Acc: 1.0. Test Acc:0.303
Epoch: 330. Train Loss: 0.5036. Test Loss: 1.283. Train Acc: 0.9804. Test Acc:0.303
Epoch: 340. Train Loss: 0.4856. Test Loss: 1.317. Train Acc: 1.0. Test Acc:0.303
Epoch: 350. Train Loss: 0.4913. Test Loss: 1.317. Train Acc: 0.9216. Test Acc:0.303
Epoch: 360. Train Loss: 0.4535. Test Loss: 1.323. Train Acc: 0.9804. Test Acc:0.303
Epoch: 370. Train Loss: 0.4345. Test Loss: 1.33. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 380. Train Loss: 0.4153. Test Loss: 1.343. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 390. Train Loss: 0.411. Test Loss: 1.356. Train Acc: 0.9412. Test Acc:0.3636
Epoch: 400. Train Loss: 0.3877. Test Loss: 1.345. Train Acc: 0.9804. Test Acc:0

Epoch: 170. Train Loss: 0.8098. Test Loss: 1.238. Train Acc: 0.8627. Test Acc:0.2727
Epoch: 180. Train Loss: 0.8133. Test Loss: 1.251. Train Acc: 0.8431. Test Acc:0.2727
Epoch: 190. Train Loss: 0.7884. Test Loss: 1.264. Train Acc: 0.8627. Test Acc:0.2424
Epoch: 200. Train Loss: 0.7603. Test Loss: 1.285. Train Acc: 0.8431. Test Acc:0.2121
Epoch: 210. Train Loss: 0.7197. Test Loss: 1.298. Train Acc: 0.8627. Test Acc:0.2121
Epoch: 220. Train Loss: 0.7377. Test Loss: 1.309. Train Acc: 0.8627. Test Acc:0.2121
Epoch: 230. Train Loss: 0.6929. Test Loss: 1.334. Train Acc: 0.8431. Test Acc:0.2121
Epoch: 240. Train Loss: 0.6689. Test Loss: 1.343. Train Acc: 0.8431. Test Acc:0.2727
Epoch: 250. Train Loss: 0.6634. Test Loss: 1.358. Train Acc: 0.9412. Test Acc:0.2727
Epoch: 260. Train Loss: 0.6503. Test Loss: 1.368. Train Acc: 0.8824. Test Acc:0.2727
Epoch: 270. Train Loss: 0.6203. Test Loss: 1.392. Train Acc: 0.8824. Test Acc:0.303
Epoch: 280. Train Loss: 0.6201. Test Loss: 1.411. Train Acc: 0.921

Epoch: 60. Train Loss: 1.036. Test Loss: 1.135. Train Acc: 0.3529. Test Acc:0.303
Epoch: 70. Train Loss: 1.01. Test Loss: 1.14. Train Acc: 0.3529. Test Acc:0.2727
Epoch: 80. Train Loss: 0.9988. Test Loss: 1.138. Train Acc: 0.3922. Test Acc:0.2727
Epoch: 90. Train Loss: 0.984. Test Loss: 1.139. Train Acc: 0.4902. Test Acc:0.2727
Epoch: 100. Train Loss: 0.9308. Test Loss: 1.146. Train Acc: 0.6667. Test Acc:0.2424
Epoch: 110. Train Loss: 0.9286. Test Loss: 1.147. Train Acc: 0.6471. Test Acc:0.2424
Epoch: 120. Train Loss: 0.9125. Test Loss: 1.147. Train Acc: 0.6863. Test Acc:0.2727
Epoch: 130. Train Loss: 0.8991. Test Loss: 1.151. Train Acc: 0.7059. Test Acc:0.2727
Epoch: 140. Train Loss: 0.8709. Test Loss: 1.151. Train Acc: 0.7451. Test Acc:0.2424
Epoch: 150. Train Loss: 0.8428. Test Loss: 1.145. Train Acc: 0.7843. Test Acc:0.2424
Epoch: 160. Train Loss: 0.8051. Test Loss: 1.15. Train Acc: 0.8039. Test Acc:0.303
Epoch: 170. Train Loss: 0.7772. Test Loss: 1.147. Train Acc: 0.9412. Test Acc

In [ ]:
LangIdRNN()

In [ ]:
raw_features.keys()